In [45]:
from scipy.spatial import distance as dst
from mpl_toolkits.basemap import Basemap
from collections import defaultdict
import matplotlib.pyplot as plt
import shapely.geometry as sp
import datetime as dt
import pandas as pd
import pickle as pk
import numpy as np
import itertools
import math
import time
import json
import copy
from scipy.spatial import distance as dst
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import shapely.geometry as sp
import distinctipy as ds
import pickle as pk
import numpy as np
import pandas as pd

<div>
<font size="4",style="font-family:Ubuntu"> <p> The following two cells demonstrates the two main csv files we will be working with in the notebook. The cleaned versions of FlightTracks.csv(cleanTracks.csv) and Flight_History.csv(cleanFlights.csv)</p></font><br>
<font size="3">
<b>Input</b> &rarr; cleanTracks.csv, cleanFlightHistory.csv <br>
</div>

In [3]:
cleanTracks=pd.read_csv("Outputs/cleanTracks.csv")
cleanTracks

,received,ground_speed,latitude_degrees,longitude_degrees,flighthistory_id
0,2013-08-14 08:30:56,172,36.207200,-79.425000,306095730
1,2013-08-14 08:31:56,187,36.259400,-79.430000,306095730
2,2013-08-14 08:32:53,214,36.326700,-79.418100,306095730
3,2013-08-14 08:33:53,191,36.363100,-79.383600,306095730
4,2013-08-14 08:34:59,186,36.402200,-79.336400,306095730
...,...,...,...,...,...
81406058,2013-09-11 13:57:30,316,42.680000,-106.449997,308962199
81406059,2013-09-11 13:58:31,279,42.700001,-106.529999,308962199
81406060,2013-09-11 13:59:32,250,42.720001,-106.620003,308962199
81406061,2013-09-11 14:24:46,130,37.630001,-122.099998,308960729


In [4]:
cleanFlights=pd.read_csv("Outputs/cleanFlightHistory.csv")
cleanFlights

,id,departure_airport_icao_code,arrival_airport_icao_code,scheduled_runway_departure,actual_runway_departure,scheduled_runway_arrival,actual_runway_arrival
0,305966493,KSDF,KDEC,2013-08-14 08:40:00,2013-08-14 09:14:00,2013-08-14 09:44:00,2013-08-14 10:12:00
1,306196254,KSDF,KBHM,2013-08-14 09:00:00,2013-08-14 09:04:00,2013-08-14 09:47:00,2013-08-14 09:51:00
2,306196711,KMEM,KCOS,2013-08-14 09:00:00,2013-08-14 09:14:00,2013-08-14 11:01:00,2013-08-14 11:18:00
3,306196678,KMEM,KMIA,2013-08-14 09:00:00,2013-08-14 09:09:00,2013-08-14 10:52:00,2013-08-14 10:58:00
4,306196521,KSDF,KMKE,2013-08-14 09:00:00,2013-08-14 09:15:00,2013-08-14 09:54:00,2013-08-14 10:04:00
...,...,...,...,...,...,...,...
646704,308962754,KMEM,KSAN,2013-09-11 08:58:00,2013-09-11 09:04:00,2013-09-11 12:07:00,2013-09-11 12:09:00
646705,308961528,KSDF,KSGF,2013-09-11 08:59:00,2013-09-11 09:08:00,2013-09-11 10:03:00,2013-09-11 10:03:00
646706,308962550,KMEM,KRIC,2013-09-11 08:59:00,2013-09-11 09:11:00,2013-09-11 10:37:00,2013-09-11 10:46:00
646707,308962750,KSDF,KHSV,2013-09-11 08:59:00,2013-09-11 09:16:00,2013-09-11 09:58:00,2013-09-11 10:26:00


<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell reads the cleanTracks.csv which has a mapping between every flight and its corresponding ground speed in knots captured at 1 minute intervals(timestamps) and outputs the dictionary as a pickle file that maps each flightID to its median ground speed/cruise speed. </p></font><br>
<font size="3">
<b>Input</b> &rarr; cleanTracks.csv <br>
<b>Output</b> &rarr; SpeedDict.pkl </font>
</div>

In [3]:
cleanTracks=pd.read_csv("Outputs/cleanTracks.csv")
cleanTracks.sort_values(by=['received'],inplace=True)
speeds = cleanTracks.groupby(["flighthistory_id"])["ground_speed"].median()
df_dict = speeds.to_dict()
avg_speed = np.array(list(df_dict.values())).mean()
med_speed = np.median(list(df_dict.values()))
print("The Average Cruise Speed of a flight is ",avg_speed)
print("The Median Cruise Speed of a flight is ",med_speed)
print("The Number of flights whose cruise speed is known",len(df_dict))
speedDictFile=open("Outputs/SpeedDict.pkl","wb")
pk.dump(df_dict,speedDictFile)
speedDictFile.close()

The Average Cruise Speed of a flight is  386.2909277154822
The Median Cruise Speed of a flight is  407.5
The Number of flights whose cruise speed is known 631131


<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell reads the cleanFlightHistory.csv and drops those flights which start and end in the same sector to avoid outliers(960 total flights dropped) </p></font><br>
<font size="3">
<b>Input</b> &rarr; cleanFlightHistory.csv, airportSectorDict.pkl <br>
<b>Output</b> &rarr; cleanFlightHistory.csv </font>
</div>

In [37]:
cleanFlights=pd.read_csv("Outputs/cleanFlightHistory.csv")
airportToSector=pk.load(open("Outputs/airportSectorDict.pkl",'rb'))
badIdx=[]
dep=cleanFlights["departure_airport_icao_code"]
arr=cleanFlights["arrival_airport_icao_code"]
NumRows=len(dep)
for idx in range(NumRows):
    if airportToSector[dep[idx]]==airportToSector[arr[idx]]:
        badIdx.append(idx)
cleanFlights.drop(badIdx, axis=0, inplace=True)
cleanFlights=cleanFlights.reset_index()
cleanFlights.drop(['index'],axis=1,inplace=True)
cleanFlights.to_csv("Outputs/cleanFlightHistory.csv",index=False)

In [38]:
cleanFlights=pd.read_csv("Outputs/cleanFlightHistory.csv")
cleanFlights

,id,departure_airport_icao_code,arrival_airport_icao_code,scheduled_runway_departure,actual_runway_departure,scheduled_runway_arrival,actual_runway_arrival
0,305966493,KSDF,KDEC,2013-08-14 08:40:00,2013-08-14 09:14:00,2013-08-14 09:44:00,2013-08-14 10:12:00
1,306196254,KSDF,KBHM,2013-08-14 09:00:00,2013-08-14 09:04:00,2013-08-14 09:47:00,2013-08-14 09:51:00
2,306196711,KMEM,KCOS,2013-08-14 09:00:00,2013-08-14 09:14:00,2013-08-14 11:01:00,2013-08-14 11:18:00
3,306196678,KMEM,KMIA,2013-08-14 09:00:00,2013-08-14 09:09:00,2013-08-14 10:52:00,2013-08-14 10:58:00
4,306196521,KSDF,KMKE,2013-08-14 09:00:00,2013-08-14 09:15:00,2013-08-14 09:54:00,2013-08-14 10:04:00
...,...,...,...,...,...,...,...
645780,308962754,KMEM,KSAN,2013-09-11 08:58:00,2013-09-11 09:04:00,2013-09-11 12:07:00,2013-09-11 12:09:00
645781,308961528,KSDF,KSGF,2013-09-11 08:59:00,2013-09-11 09:08:00,2013-09-11 10:03:00,2013-09-11 10:03:00
645782,308962550,KMEM,KRIC,2013-09-11 08:59:00,2013-09-11 09:11:00,2013-09-11 10:37:00,2013-09-11 10:46:00
645783,308962750,KSDF,KHSV,2013-09-11 08:59:00,2013-09-11 09:16:00,2013-09-11 09:58:00,2013-09-11 10:26:00


<h1>Test Days</h1><br>
<font size="4",style="font-family:Ubuntu"> <p> The days tested for are given below - we obtain the 3 metrics for each of the test days and compare the metric with the predicted value from the GA <br><br> Range = [2013-08-14,2013-09-11]</p></font><br>

In [5]:
TEST_DAYS=["2013-08-16"]
NumberOfHours=1

<p><font size="4",style="font-family:Ubuntu"> <p> The cell below creates seperate .csv files for the tracks and flights of each test day in order to reduce the IO burden, as we would not want to read the entire csv files(cleanFlightHistory and cleanTracks) for the whole month since we just want to analyze a single test day </p>
<b>Input</b> &rarr; cleanTracks.csv,cleanFlightHistory.csv <br>
<b>Output</b> &rarr; 2N csv files where N is the number of test days, for each test day we generate 2 csv files, one containing all the flights scheduled to depart that day({date}_flights.csv) and all the flight tracks recorded that day ({date}_tracks.csv) </font></font><br>

In [6]:
cleanTracks=pd.read_csv("Outputs/cleanTracks.csv")
cleanFlights=pd.read_csv("Outputs/cleanFlightHistory.csv")
for date in TEST_DAYS:
    tracktemp=cleanTracks[cleanTracks['received'].str.contains(date)].copy()
    flighttemp=cleanFlights[cleanFlights['scheduled_runway_departure'].str.contains(date)].copy()
    tracktemp.to_csv(f"MetricFiles/DayWiseInput/{date}_tracks.csv",index=False)
    flighttemp.to_csv(f"MetricFiles/DayWiseInput/{date}_flights.csv",index=False)

<h1>Metric 1 - Traffic Factor</h1>

<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell block reads the {test_day}_tracks.csv which has a mapping between every flight and the coordinates of its actual path measured at constant time intervals. We use this data to obtain the TrafficFactor for each test day.<b> The Traffic Factor is a measure of the maximum number of aircraft present in every sector (measure the sector count of each sector) at any given time across the entire day.</b> </p></font><br>
<font size="3">
<b>Input</b> &rarr; {test_day}_tracks.csv <br>
<b>Output</b> &rarr; N pickle files where N is the number of days to be tested. &rarr; Each one a dict mapping each Sector to its traffic factor </font>
</div>

In [4]:
def getChunk(Point,SectorChunkDict):
    innerDict=SectorChunkDict[math.ceil(Point[0]/1000000)*1000000]
    Y=0
    for key in innerDict.keys():
        if key >= Point[1]:
            Y=key
            break
    return innerDict[Y] if Y!=0 else []
def findSectorPath(path,sectorPath,z,SectorChunkDict,output):
    sector=-1
    for pointIdx in range(len(path)):
        sectorList=getChunk(path[pointIdx],SectorChunkDict)
        for i in sectorList:
            if len(output[i]) < 3:
                continue
            if sp.Polygon(output[i]).contains(sp.Point(path[pointIdx])):
                sector=i
                break
        if sector!=-1:
            sectorPath.append(sector)
        else:
            z.append(pointIdx)
def getSectorTimings(dayTracks, m, SectorTimeDict, SectorChunkDict, ConvexHulls):
    dayTracks.sort_values(by=['flighthistory_id','received'],inplace=True)
    dayTracks=dayTracks.reset_index()
    dayTracks.drop(['index'],axis=1,inplace=True)
    east,north=m(dayTracks["longitude_degrees"],dayTracks["latitude_degrees"])
    path=list(zip(east,north))
    sectorPath=[]
    z=[]
    findSectorPath(path,sectorPath,z,SectorChunkDict,ConvexHulls)
    dayTracks.drop(z, axis=0, inplace=True)
    dayTracks=dayTracks.reset_index()
    dayTracks.drop(['index'],axis=1,inplace=True)
    timings=list(map(lambda x:dt.datetime.fromisoformat(str(x)),dayTracks["received"]))
    rowIdx=0
    NumRows=len(sectorPath)
    while rowIdx<NumRows:
        sector=sectorPath[rowIdx]
        startTime=(timings[rowIdx].hour*60)+timings[rowIdx].minute
        while rowIdx<NumRows and sectorPath[rowIdx]==sector:
            rowIdx+=1
        endTime=(timings[rowIdx-1].hour*60)+timings[rowIdx-1].minute
        for minute in range(startTime,endTime+1):
            SectorTimeDict[sector][minute]+=1

In [7]:
ConvexHulls = pk.load(open("Outputs/ConvexDict.pkl", "rb"))
m = pk.load(open("Outputs/M_ConversionMetric.pkl", "rb"))
SectorChunkDict=pk.load(open("Outputs/SectorChunkDict.pkl", "rb"))
NumSectors=1250
TimeUnits=1440
for date in TEST_DAYS:
    SectorTimeDict=[[0 for colnum in range(TimeUnits)] for rownum in range(NumSectors)]
    dayTracks=pd.read_csv(f"MetricFiles/DayWiseInput/{date}_tracks.csv")
    getSectorTimings(dayTracks,m,SectorTimeDict,SectorChunkDict,ConvexHulls)
    filehandler = open(f"MetricFiles/{date}_SectorTimeDict.pkl","wb")
    pk.dump(SectorTimeDict,filehandler)
    filehandler.close()
    TrafficFactor=dict()
    for Sectoridx in range(NumSectors):
        TrafficFactor[Sectoridx]=max(SectorTimeDict[Sectoridx])
    filehandler = open(f"MetricFiles/{date}_TrafficFactor.pkl","wb")
    pk.dump(TrafficFactor,filehandler)
    filehandler.close()

<h1>Metric 2 - Aerial Time</h1>
<h1>Metric 3 - Ground Delay</h1>

<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell block reads the {test_day}_flights.csv which details every single flight that was scheduled to depart on {test_day} We use this dataset to obtain the <b>actual aerial time </b> based on the difference between actual runway arrival and actual runway departure and the <b> ground holding delay time </b> based on the difference between actual runway departure and scheduled runway departure</p></font><br>
<font size="3">
<b>Input</b> &rarr; {test_day}_flights.csv <br>
<b>Output</b> &rarr; N csv files &rarr; each contains 2 columns, aerial time and ground delay </font>
</div>

In [53]:
for day in TEST_DAYS:
    dayDf=pd.read_csv(f"MetricFiles/DayWiseInput/{day}_flights.csv")
    dayDf.sort_values(by=["scheduled_runway_departure"], inplace = True)
    ground_delay = []
    actual_air_time = []
    srd=list(map(lambda x:dt.datetime.fromisoformat(str(x)),dayDf["scheduled_runway_departure"]))
    ard=list(map(lambda x:dt.datetime.fromisoformat(str(x)),dayDf["actual_runway_departure"]))
    ara=list(map(lambda x:dt.datetime.fromisoformat(str(x)),dayDf["actual_runway_arrival"]))
    NumRows=len(ard)
    for i in range(NumRows):
        ground_delay.append((ard[i] - srd[i])/pd.Timedelta(minutes=1))
        actual_air_time.append((ara[i]-ard[i])/pd.Timedelta(minutes=1))
    dayno_data = pd.DataFrame()
    dayno_data.insert(0, "ground_delay", ground_delay)
    dayno_data.insert(1, "aerial-time", actual_air_time)
    dayno_data.to_csv(f"MetricFiles/{day}-gd-aat.csv",index=False)

<h1>Input File Generation</h1>

<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell generates the test files which will serve as an input to the GA via the website</p></font><br>
<font size="3">
<b>Input</b> &rarr; {test_day}_flights.csv <br>
<b>Output</b> &rarr; N txt files &rarr; each serves as a single days' worth of flights to be input to the model  </font>
</div>

In [54]:
for day in TEST_DAYS:
    dayDf=pd.read_csv(f"MetricFiles/DayWiseInput/{day}_flights.csv")
    dayDf.sort_values(by=["scheduled_runway_departure"], inplace = True)
    ids=list(dayDf['id'])
    source=list(dayDf["departure_airport_icao_code"])
    destination=list(dayDf["arrival_airport_icao_code"])
    startTime=[dt.datetime.fromisoformat(str(date)) for date in dayDf["scheduled_runway_departure"]]
    triplets=list(zip(ids,source,destination))
    start=[]
    for s in startTime:
        hour=s.hour
        minute=s.minute
        if minute < 10:
            minute="0" + str(minute)
        start.append(str(hour) + ":" + str(minute))
    toPrint=""
    for i in range(len(triplets)):
        flightID,sourceAirport,destinationAirport=triplets[i]
        sourceAirportName=AirportDB[sourceAirport]["name"]
        destinationAirportName=AirportDB[destinationAirport]["name"]
        if flightID not in speedDict:
            speedDict[flightID]=407.5
        toPrint+=f"{sourceAirportName},{destinationAirportName},1,{start[i]},{speedDict[flightID]}"
        if i!=len(triplets)-1:
            toPrint+="\n"
    f = open(f"MetricFiles/Input Text Files/{day}-pairs.txt", "w")
    f.write(toPrint)
    f.close()